In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
# SETUP EVALUATORS
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2578 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2164 (17.1%) Users that have less than 1 test interactions


In [3]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender


MAP_recommender_per_group = {}

collaborative_recommender_class = {"UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "SLIM_ELASTIC": MultiThreadSLIM_SLIMElasticNetRecommender,
                                   #"EASE_R": EASE_R_Recommender,
                                   }
collaborative_recommender_class.items()

recommender_object_dict = {}

for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train_validation)
    recommender_object_dict[label] = recommender_object
    
    
recommender_object_dict["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=7)
recommender_object_dict["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict["RP3beta"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
#recommender_object_dict["EASE_R"].fit(topK=24, l2_norm=37.54323189430143)

UserKNNCFRecommender: URM Detected 207 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 99 ( 0.4%) items with no interactions.
ItemKNNCFRecommender: URM Detected 207 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 99 ( 0.4%) items with no interactions.
P3alphaRecommender: URM Detected 207 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 99 ( 0.4%) items with no interactions.
RP3betaRecommender: URM Detected 207 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 99 ( 0.4%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 207 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 99 ( 0.4%) items with no interactions.


100%|█████████▉| 22192/22222 [01:44<00:00, 134.56it/s]

P3alphaRecommender: Similarity column 22222 (100.0%), 5373.18 column/sec. Elapsed time 4.14 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 4284.52 column/sec. Elapsed time 5.19 sec
Similarity column 12638 (100.0%), 8951.99 column/sec. Elapsed time 1.41 sec


100%|█████████▉| 22216/22222 [02:00<00:00, 134.56it/s]

Similarity column 22222 (100.0%), 8295.28 column/sec. Elapsed time 2.68 sec


In [16]:
import  Recommenders.XGBoostTrain as XGBoostTrain
from importlib import reload
reload(XGBoostTrain)

<module 'Recommenders.XGBoostTrain' from '/home/alessandro/RecSys-2023-polimi/Recommenders/XGBoostTrain.py'>

In [17]:
from xgboost import XGBRanker
from Recommenders.XGBoostTrain import XGBoostTrain

import optuna as op
import xgboost as xgb

model = XGBoostTrain(URM_train_validation, URM_val=URM_test, recommenders=recommender_object_dict, XGB_model=None)



def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "rank:pairwise",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear"]),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    xgb_model = XGBRanker(**param)
    # XGBRanker(
    #     objective='rank:pairwise',
    #     n_estimators=1000,
    #     learning_rate=0.19823429576094637,
    #     reg_alpha=47,
    #     reg_lambda=0.313,
    #     max_depth=3,
    #     verbosity=0,
    #     booster="gbtree",
    #     colsample_bytree=0.6113704247857885,
    #     gamma=8.964184693722684,
    #     min_child_weight=7.0,
    #     tree_method="hist"
    # )
    # Add a callback for pruning.
    #pruning_callback = op.integration.XGBoostPruningCallback(trial, "validation-auc")
    model.fit(XGB_model=xgb_model)
        
    result, _ = evaluator_test.evaluateRecommender(model)
    MAP_result = result["MAP"].item()
       
    return MAP_result

  1%|          | 141/12638 [00:00<00:08, 1402.49it/s]

XGBoostTrain: URM Detected 207 ( 1.6%) users with no interactions.
XGBoostTrain: URM Detected 99 ( 0.4%) items with no interactions.
CREATING THE DATAFRAME


  0%|          | 12/12638 [00:00<01:51, 113.34it/s]

POPULATING THE DATAFRAME


100%|██████████| 12638/12638 [01:39<00:00, 126.64it/s]


DF LOADING END


In [ ]:
best_params = {
    
}

study_name = "XGB-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize",pruner=op.pruners.MedianPruner(n_warmup_steps=5), load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=10)

[I 2023-12-06 00:58:41,807] Using an existing study with name 'XGB-study' instead of creating a new one.
 16%|█▋        | 163/1000 [00:00<00:00, 1622.89it/s]

CREATING THE DATAFRAME


  3%|▎         | 26/1000 [00:00<00:03, 257.30it/s]

POPULATING THE DATAFRAME


 17%|█▋        | 173/1000 [00:00<00:00, 1722.36it/s]

...

 17%|█▋        | 168/1000 [00:00<00:00, 1672.16it/s]

CREATING THE DATAFRAME


  2%|▏         | 22/1000 [00:00<00:04, 218.45it/s]

POPULATING THE DATAFRAME


 58%|█████▊    | 578/1000 [00:02<00:01, 260.68it/s]

In [16]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2184 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10454 (100.0%) in 8.71 sec. Users per second: 1200


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.089028                 0.150759  0.12639  0.04558    0.076747   
 
             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.26035  0.140004  0.104469  0.493495      0.340601  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.408213    0.827188       0.057399   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.156523                   0.997471             0.164554   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                            

In [17]:
#final.fit(alphas=[3,1.25])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Processed 10454 (100.0%) in 6.85 sec. Users per second: 1526


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.079759                 0.137157  0.11639  0.03848    0.066675   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.234882  0.125152  0.094654  0.475703      0.298827  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.393496    0.827188       0.050161   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.020847                   0.997362             0.145638   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                         